# Tensorflow Models

## mnist

In [ ]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

learning_rate = 0.01
batch_size = 32
n_epochs = 30

In [ ]:
mnist = input_data.read_data_sets('data/mnist', one_hot=True)
X_batch, Y_batch = mnist.train.next_batch(batch_size)

In [ ]:
print('learning_rate ', learning_rate)
print('batch_size ', batch_size)
print('n_epochs ', n_epochs)
print('X_batch', X_batch.shape, X_batch.dtype)
print('Y_batch', Y_batch.shape, X_batch.dtype)

image_size = X_batch.shape[1]
n_classes = Y_batch.shape[1]

X = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size], name='input')
Y = tf.placeholder(dtype=tf.int32, shape=[batch_size, n_classes], name='label')

n_hiden_layer = 10

w = tf.get_variable(name='weights', dtype=tf.float32, shape=[image_size, n_hiden_layer], initializer=tf.truncated_normal_initializer())
b = tf.get_variable(name='bias', dtype=tf.float32, shape=[1, n_hiden_layer], initializer=tf.truncated_normal_initializer())

logits = tf.matmul(X, w) + b
print('logits ',logits.shape)

# train
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
print('entropy ', entropy.shape)

loss = tf.reduce_mean(entropy)
print('loss ', loss.shape)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# test
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./graphs/logreg_placeholder', tf.get_default_graph())

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_train_batches = int(mnist.train.num_examples / batch_size)
    n_test_batches = int(mnist.test.num_examples / batch_size)

    for i in range(n_epochs):
        total_loss = 0

        # train
        for j in range(n_train_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch, accuracy_batch = sess.run([optimizer, loss, accuracy], {X: X_batch, Y: Y_batch})
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}: {2}'.format(i, total_loss / n_train_batches, accuracy_batch / batch_size))

        # validation
        total_correct_preds = 0
        for j in range(n_test_batches):
            X_batch, Y_batch = mnist.test.next_batch(batch_size)
            accuracy_batch = sess.run(accuracy, {X: X_batch, Y: Y_batch})
            total_correct_preds += accuracy_batch
        print('Epoch {0} Accuracy {1}'.format(i, total_correct_preds / mnist.test.num_examples))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test
    total_correct_preds = 0

    for i in range(n_test_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, {X: X_batch, Y: Y_batch})
        total_correct_preds += accuracy_batch

    print('Total: Accuracy {0}'.format(total_correct_preds / mnist.test.num_examples))



## word2vec

In [2]:
import gzip
import os
import random
import shutil
import urllib
import zipfile
from collections import Counter

import numpy as np
import tensorflow as tf

VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128  # dimension of the word embedding vectors
SKIP_WINDOW = 1  # the context window
NUM_SAMPLED = 64  # number of negative examples to sample
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
VISUAL_FLD = 'word2vec'
SKIP_STEP = 5000

DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000  # number of tokens to visualize

In [3]:
def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass
    
def download_one_file(download_url,
                      local_dest,
                      expected_byte=None,
                      unzip_and_remove=False):
    if os.path.exists(local_dest) or os.path.exists(local_dest[:-3]):
        print('%s already exists' % local_dest)
    else:
        print('Downloading %s' % download_url)
        local_file, _ = urllib.request.urlretrieve(download_url, local_dest)
        file_stat = os.stat(local_dest)
        if expected_byte:
            if file_stat.st_size == expected_byte:
                print('Successfully downloaded %s' % local_dest)
                if unzip_and_remove:
                    with gzip.open(local_dest, 'rb') as f_in, open(local_dest[:-3], 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    os.remove(local_dest)
            else:
                print('The downloaded file has unexpected number of bytes')


def read_data(file_path):
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return words


def build_vocab(words, vocab_size, visual_fld):
    safe_mkdir(visual_fld)
    file = open(os.path.join(visual_fld, 'vocab.tsv'), 'w')

    dictionary = dict()
    count = [('UNK', -1)]
    index = 0
    count.extend(Counter(words).most_common(vocab_size - 1))

    for word, _ in count:
        dictionary[word] = index
        index += 1
        file.write(word + '\n')

    index_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    file.close()
    return dictionary, index_dictionary


def generate_sample(index_words, context_window_size):
    """ Form training pairs according to the skip-gram model. """
    for index, center in enumerate(index_words):
        context = random.randint(1, context_window_size)
        # get a random target before the center word
        for target in index_words[max(0, index - context): index]:
            yield center, target
        # get a random target after the center wrod
        for target in index_words[index + 1: index + context + 1]:
            yield center, target


def generator(download_url, expected_byte, vocab_size, batch_size, skip_window, visual_fld):
    local_dest = 'data/text8.zip'
    download_one_file(download_url, local_dest, expected_byte)
    words = read_data(local_dest)
    dictionary, index_dictionary = build_vocab(words, vocab_size, visual_fld)
    index_words = [dictionary[word] if word in dictionary else 0 for word in words]
    del words  # to save memory
    single_gen = generate_sample(index_words, skip_window)

    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1])
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(single_gen)
        yield center_batch, target_batch